# 1.Basic Model Preprocessing with Dynamic padding:

In [ ]:
!pip install datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset

checkpoint = 'bert-base-uncased'
tokenizer= AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)

input = load_dataset('glue','stsb')

def token_fun(lines):
  return(tokenizer(lines['sentence1'],lines['sentence2'], truncation =True))

data_collator = DataCollatorWithPadding(tokenizer)

tokenized_data=input.map(token_fun,  batched = True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

# 2.Initializing Training arguments, Trainer and Training the model

In [ ]:
from transformers import TrainingArguments, Trainer

train_args = TrainingArguments('train_model',
                               per_device_train_batch_size=16,
                               per_device_eval_batch_size=16,
                               num_train_epochs=5,
                               learning_rate=2e-5,
                               weight_decay=0.01
                               )
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer
)
trainer.train()

<ipython-input-8-2f8ecded7fcd>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pavansaigundaram14 (pavansaigundaram14-university-of-north-texas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
500,0.949800
1000,0.340600
1500,0.203400


TrainOutput(global_step=1800, training_loss=0.4391308911641439, metrics={'train_runtime': 415.8761, 'train_samples_per_second': 69.119, 'train_steps_per_second': 4.328, 'total_flos': 917696913063630.0, 'train_loss': 0.4391308911641439, 'epoch': 5.0})

In [ ]:
predictions = trainer.predict(tokenized_data['validation'])
print(predictions.predictions.shape, predictions.label_ids.shape)

(1500, 1) (1500,)


In [ ]:
!pip install evaluate


In [ ]:
import numpy as np
from evaluate import load

#from datasets import load_metric
metric=load('glue','stsb')
preds = predictions.predictions[:, 0]
labels = predictions.label_ids
metric.compute(predictions=preds,references=labels)

{'pearson': 0.8935132737776627, 'spearmanr': 0.8886999978424391}

**Pearson Correlation (pearson)**

Measures the linear relationship between predicted similarity scores and ground-truth scores.
Ranges from -1 (perfect negative correlation) to 1 (perfect positive correlation).
If all values in preds or labels are constant, Pearson correlation is undefined (NaN).


**Spearman Correlation (spearmanr)**

Measures the rank correlation (monotonic relationship) between predictions and ground truth.
More robust than Pearson for non-linear relationships.
Also ranges from -1 to 1, and will be NaN if inputs are constant.

# Complete Training the Model:

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from evaluate import load
from datasets import load_dataset


input= load_dataset('glue','stsb')

checkpoint = 'bert-base-uncased'

tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)

def token_fun(lines):
  return tokenizer(lines['sentence1'],lines['sentence2'],truncation=True)

data_collator = DataCollatorWithPadding(tokenizer)

model_input = input.map(token_fun,batched=True)

def compute_metric(eval_preds):
  logits, labels = eval_preds.predictions, eval_preds.label_ids
  metric= load('glue','stsb')
  result=metric.compute(predictions=logits[:,0], references=labels)
  return result


train_args=TrainingArguments('training model',
                             per_device_train_batch_size=16,
                             per_device_eval_batch_size=16,
                             num_train_epochs=5,
                             learning_rate=2e-5,
                             weight_decay=0.01,
                             evaluation_strategy="epoch"
                             )

trainer=Trainer(
    model=model,
    args=train_args,
    train_dataset=model_input['train'],
    eval_dataset=model_input['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metric
)

trainer.train()



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-21-b4f5f23428d7>:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(


Epoch,Training Loss,Validation Loss,Pearson,Spearmanr
1,No log,0.503134,0.883326,0.881174
2,0.864300,0.560891,0.887154,0.884669
3,0.310700,0.501990,0.892894,0.888669
4,0.310700,0.477969,0.893455,0.889533
5,0.180100,0.477199,0.894623,0.890584


TrainOutput(global_step=1800, training_loss=0.39735864215426975, metrics={'train_runtime': 475.3196, 'train_samples_per_second': 60.475, 'train_steps_per_second': 3.787, 'total_flos': 917696913063630.0, 'train_loss': 0.39735864215426975, 'epoch': 5.0})